For a dataset like MNIST, what do you think our GAN generator would produce? Do you think it will invent images that look like digits but aren’t? Or will it simply create variations on the digits 0 through 9? Run the GAN code on MNIST digits to see. Tip: Using GPUs will greatly speed up the experiment. 


Import key libraries.

In [36]:
from keras.preprocessing import image
import numpy as np

Collect a set of 1000 MNIST images, resized to 36 by 36 like the daisy images.

In [37]:
# read in images:

from keras.datasets import mnist

def load_mnist_data():
    # load the data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # normalize our inputs to be in the range[-1, 1]
#    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    # convert x_train with a shape of (60000, 28, 28) to (60000, 784) so we have
    # 784 columns per row
    x_train = x_train.reshape(60000, 784)
    return (x_train)

x_train = load_mnist_data()

daisies = []
for x in x_train[:1000]:
    img = image.array_to_img(np.concatenate((x.reshape(28,28,1),x.reshape(28,28,1),x.reshape(28,28,1)),axis=2))
    img = img.resize((36,36))
    img_arr = image.img_to_array(img)
    img_arr = img_arr.flatten()
    img_arr = img_arr.reshape(36,36,3)
    daisies += [img_arr]

Create a place for the generated images.

In [42]:
!mkdir images

Run the daisy GAN code.

In [43]:
# from https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py

from __future__ import print_function, division

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 36
        self.img_cols = 36
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        quartersize = 9 # 16 # 7
        model.add(Dense(128 * quartersize * quartersize, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((quartersize, quartersize, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

#        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

#        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
#        (X_train, _), (_, _) = mnist.load_data()
        X_train = np.array(daisies)
#        print(X_train.shape)

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
#        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
#            print(gen_imgs.shape)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/daisies_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    dcgan = DCGAN()
    dcgan.train(epochs=4001, batch_size=32, save_interval=50) # 4001
#    dcgan.train(epochs=501, batch_size=32, save_interval=50)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.119082, acc.: 39.06%] [G loss: 0.701800]
1 [D loss: 0.637556, acc.: 65.62%] [G loss: 1.196018]
2 [D loss: 0.343461, acc.: 85.94%] [G loss: 1.351221]
3 [D loss: 0.266733, acc.: 90.62%] [G loss: 1.364596]
4 [D loss: 0.179197, acc.: 98.44%] [G loss: 1.261677]
5 [D loss: 0.188232, acc.: 96.88%] [G loss: 1.405647]
6 [D loss: 0.179625, acc.: 93.75%] [G loss: 1.307706]
7 [D loss: 0.236228, acc.: 93.75%] [G loss: 1.058155]
8 [D loss: 0.151618, acc.: 95.31%] [G loss: 1.530633]
9 [D loss: 0.172757, acc.: 98.44%] [G loss: 1.646605]
10 [D loss: 0.273970, acc.: 92.19%] [G loss: 1.566303]
11 [D loss: 0.266646, acc.: 89.06%] [G loss: 1.398467]
12 [D loss: 0.246802, acc.: 92.19%] [G loss: 1.556997]
13 [D loss: 0.208638, acc.: 95.31%] [G loss: 1.782079]
14 [D loss: 0.258737, acc.: 90.62%] [G loss: 1.594288]
15 [D loss: 0.223269, acc.: 96.88%] [G loss: 1.516419]
16 [D loss: 0.207570, acc.: 92.19%] [G loss: 1.671719]
17 [D loss: 0.392360, acc.: 87.50%] [G loss: 1.673794]
18 [D loss: 0.244436

Display the evolution of the generated images.

In [46]:
from keras.preprocessing import image

for e in range(int(4000/50)+1):
  print(e*50)
  img = image.load_img("images/daisies_"+str(e*50)+".png")
  display(img)

Output hidden; open in https://colab.research.google.com to view.